In [1]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import mean_squared_error as mse

from datasets import get_dataset, list_datasets, train_test_split
from zadania import DecisionTree

In [2]:
balance = get_dataset("balance-scale")
banknote = get_dataset("banknote")
iris = get_dataset("iris")
wine = get_dataset("wine")
car = get_dataset("car")

for name in list_datasets():
    print(name, get_dataset(name).dtypes)

balance-scale {'X': 'categorical (ordered)', 'y': 'categorical (ordered)'}
banknote {'X': 'continuous', 'y': 'categorical'}
car {'X': 'categorical (ordered)', 'y': 'categorical (ordered)'}
iris {'X': 'continuous', 'y': 'categorical'}
wine {'X': 'mixed', 'y': 'continuous'}


Domyślne parametry sklearn'a:
* `DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)`
* `DecisionTreeRegressor(criterion=’mse’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, presort=False)`

In [3]:
class DTR:
    def __init__(self, X, y):
        self.m = DecisionTreeRegressor()
        self.m.fit(X, y)
    def predict(self, X):
        return self.m.predict(X)

class DTC:
    def __init__(self, X, y, criterion, max_depth=None, min_samples_split=2):
        self.m = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split)
        self.m.fit(X, y)
    def predict(self, X):
        return self.m.predict(X)

### Co należy przede wszystkim sprawdzić

#### Wynik na zbiorze treningowym

Jeśli podczas uczenia nie stosujemy pruningu i drzewo budowane jest do samego końca, to wynik na zbiorze treningowym musi wynosić 100% dla accuracy i 0. dla MSE, jeśli tylko w zbiorze treningowym nie ma dwóch identycznych x'ów o różnych y'ach (dlaczego?).

#### Liczba liści, rozmiar liści, głębokość drzewa

Dobrze jest dodać w `__init__` asserty, które po nauczeniu sprawdzają, czy:
* głębokość drzewa nie przekracza maksymalnej dozwolonej,
* suma liczb elementów we wszystkich liściach jest równa rozmiarowi zbioru treningowego,
* nie ma pustych liści,
* drzewo jest binarne (n_węzłów = n_liści - 1), chyba że robimy niebinarne splity, czego robić nie polecam,
* rozmiar żadnego liścia nie przekracza maksymalnego rozmiaru.

#### Kryterium splitu binarnego

Jeśli w węźle jest $n$ danych, to należy sprawdzić $n-1$ (nie $n$) splitów. Nie może się okazać, że najlepszy split jest gorszy od braku splitu (uważać na wzory z wariancją!). Nie można podzielić node'a o rozmiarze $n$ na dwa node'y o rozmiarach odpowiednio $n$ oraz $0$.

Jeśli w węźle są dwa identyczne wiersze feature'ów $x_1 = x_2$, to nie ma jak wykonać na nich splitu, nawet jeśli $y_1 \neq y_2$ i ten fakt trzeba uwzględnić przy sprawdzaniu warunków utworzenia liścia!

Jeśli w węźle wszystkie $y$ są identyczne, to teoretycznie można dalej go splitować, ale otrzymane w ten sposób drzewo będzie __równoważne__ (dlaczego?) drzewu, w którym przerywamy splitowanie i tworzymy liść.

#### Wartości domyślne cech w drzewach uczonych na Categorical Dataset

Jeśli features w tablicy `X` są typu categorical/discrete, a nie continuous, to w zbiorze testowym mogą pojawić się __wartości__ pewnej cechy - niech dla ustalenia uwagi będzie to pierwsza kolumna `X` - które nie wystąpiły ani razu w zbiorze treningowym. W takim wypadku jeśli podczas predykcji dotrzemy do node'a, który wykonuje split po pierwszej kolumnie, to drzewo nie będzie wiedziało, jak zaklasyfikować dany przykład.

Z tego powodu drzewa uczone na danych categorical powinny mieć w każdym węźle zdefiniowaną ścieżkę domyślną - najlepiej (chyba) wybrać tę ścieżkę, która miała najwięcej przykładów w zbiorze treningowym, czyli w pewnym sensie jest najbardziej prawdopodobna.

Rozwiązanie z domyślną ścieżką jest o tyle dobre, że być może w dalszej części drzewa zostaną wzięte pod uwagę pozostałe cechy danego przykładu.

Dlaczego ten problem nie zachodzi dla danych `X` typu continuous?

In [4]:
def eval(model_cls, model_kwargs, dataset, fn_score):
    split = train_test_split(dataset)
    model = model_cls(split.train.X, split.train.y, **model_kwargs)
    score = {
        "train": fn_score(split.train.y, model.predict(split.train.X)),
        "test": fn_score(split.test.y, model.predict(split.test.X)),
    }
    return {
        "split": split,
        "model": model,
        "score": score
    }

In [5]:
# GINI VS ENTROPY ON CONTINUOUS FEATURES

#dataset = balance
dataset = banknote
#dataset = car
#dataset = iris

print("sklearn DTC with 'gini':", eval(DTC, {"max_depth": None, "criterion": "gini"}, dataset, acc)["score"])
print("sklearn DTC with 'entropy':", eval(DTC, {"max_depth": None, "min_samples_split": 2, "criterion": "entropy"}, dataset, acc)["score"])
evaluated = eval(DecisionTree, {"max_depth": None, "min_size": 2}, dataset, acc)
print("DecisionTree:", evaluated["score"])
print("Tree structure:")
print(evaluated["model"])

sklearn DTC with 'gini': {'train': 1.0, 'test': 0.98783454987834551}
sklearn DTC with 'entropy': {'train': 1.0, 'test': 0.99026763990267641}
DecisionTree: {'train': 1.0, 'test': 0.97810218978102192}
Tree structure:
 f: 0 thr: 0.30081 pred: 0
- f: 1 thr: 5.8333 pred: 1
-- f: 2 thr: 2.9986 pred: 1
--- pred: 1
--- f: 1 thr: -1.8624 pred: 1
---- f: 0 thr: -0.69879 pred: 1
----- pred: 1
----- f: 0 thr: -0.64472 pred: 1
------ pred: 0
------ pred: 1
---- f: 0 thr: -2.5919 pred: 0
----- pred: 1
----- pred: 0
-- f: 0 thr: -4.2249 pred: 0
--- pred: 1
--- pred: 0
- f: 0 thr: 2.3917 pred: 0
-- f: 2 thr: -1.7859 pred: 0
--- f: 1 thr: 4.8731 pred: 1
---- pred: 1
---- pred: 0
--- f: 0 thr: 0.75896 pred: 0
---- f: 3 thr: -0.068117 pred: 0
----- f: 2 thr: -1.4501 pred: 0
------ f: 0 thr: 0.40614 pred: 0
------- pred: 1
------- pred: 0
------ pred: 0
----- f: 2 thr: 1.7048 pred: 1
------ pred: 1
------ pred: 0
---- pred: 0
-- pred: 0


In [5]:
# VARIANCE ON CONTINUOUS FEATURES

dataset = wine

print("sklearn DTR:", eval(DTR, {}, dataset, mse)["score"])
evaluated = eval(DecisionTree, {"max_depth": None, "min_size": 2}, dataset, mse)
print("DecisionTree:", evaluated["score"])
print("Tree structure:")
print(evaluated["model"])

sklearn DTR: {'train': 0.0, 'test': 50777.981132075474}
DecisionTree: {'train': 0.0, 'test': 56809.037735849059}
Tree structure:
 f: 0 thr: 1.0 pred: 735.576
- f: 10 thr: 6.2 pred: 1119.41025641
-- f: 1 thr: 13.28 pred: 1025.0
--- f: 2 thr: 2.05 pred: 827.857142857
---- f: 1 thr: 12.85 pred: 902.5
----- pred: 1015.0
----- f: 2 thr: 1.77 pred: 865.0
------ f: 1 thr: 13.05 pred: 882.5
------- pred: 885.0
------- pred: 880.0
------ pred: 830.0
---- f: 2 thr: 3.8 pred: 728.333333333
----- f: 1 thr: 12.93 pred: 752.5
------ pred: 770.0
------ pred: 735.0
----- pred: 680.0
--- f: 12 thr: 3.58 pred: 1087.72727273
---- f: 3 thr: 2.41 pred: 1115.25
----- f: 5 thr: 94.0 pred: 1035.5
------ f: 1 thr: 13.3 pred: 1217.5
------- pred: 1285.0
------- pred: 1150.0
------ f: 1 thr: 13.56 pred: 990.0
------- f: 1 thr: 13.48 pred: 857.5
-------- pred: 920.0
-------- pred: 795.0
------- f: 1 thr: 13.9 pred: 1034.16666667
-------- f: 2 thr: 1.67 pred: 1011.66666667
--------- pred: 1060.0
--------- f: 1 thr